In [1]:
import ollama
from ollama import chat
from ollama import ChatResponse
import time
import datetime
import tiktoken
import re
import kagglehub
import os
import pandas as pd

In [2]:
def ask_deepseek(input_content, system_prompt, deep_think = True, print_log = True):
    response: ChatResponse = chat(model="huihui_ai/deepseek-r1-abliterated:14b-qwen-distill",
                                  messages=[
                                      {'role' : 'system','content' : system_prompt},
                                      {'role' : 'user','content' : input_content}
                                      ]
                                 )
    response_text = response['message']['content']
    if print_log: print(response_text)
    #Extract and separate Deepthink:
    think_texts = re.findall(r'<think>(.*?)</think>', response_text, flags=re.DOTALL)
    think_texts = "\n\n".join(think_texts).strip()
    #Exclude Deep Think and return response
    clean_response = re.sub(r'<think>.*?</think>','', response_text, flags=re.DOTALL).strip()
    #Return either response or tuple with response and deep think
    return clean_response if not deep_think else (clean_response, think_texts)

In [3]:
path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")

In [4]:
data_labeled = pd.read_csv(path+'\\'+'all-data.csv', encoding='latin1', header=None, names=['sentiment', 'headline'])

In [5]:
data_labeled

,sentiment,headline
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [6]:
headline_data = pd.DataFrame(data_labeled['headline'])

In [7]:
headline_data

,headline
0,"According to Gran , the company has no plans t..."
1,Technopolis plans to develop in stages an area...
2,The international electronic industry company ...
3,With the new production plant the company woul...
4,According to the company 's updated strategy f...
...,...
4841,LONDON MarketWatch -- Share prices ended lower...
4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,Operating profit fell to EUR 35.4 mn from EUR ...
4844,Net sales of the Paper segment decreased to EU...


In [ ]:
system_prompt_sentiment = '''
You are a professional financial analyst in charge of guiding investment decisions based on news headlines.
You will be provided with several headlines. For each headline, provide the following:

<ID=n>
Label: [positive | negative | neutral]
<think>One-sentence explanation for your decision.</think>

Only use this format. Repeat for each ID.
'''

#feed headlines to deepseek using apply
headline_data[['LLM_sentiment', 'LLM_deep_think']] = headline_data['headline'].head(200).apply(lambda x: ask_deepseek(x,
                                                                                                            system_prompt_sentiment,
                                                                                                            print_log=False)).apply(pd.Series)